### Import CSV files to a postgreSQL database - automated

### Contents<a class="anchor" id="contents"></a>

- [1. Isolate csv files in cwd](#section1)
- [2. Create a new directory in cmd](#section2)
- [3. Move csv files into new directory](#section3)
- [4. Create pandas DataFrame from csv file(s)](#section4)
- [5. Import CSV files to a postgreSQL database - automated script](#section5)
    - [5-1. Quick reference](#section5.1)

In [1]:
## Import libraries
import os
import numpy as np
import pandas as pd
## Enables access to PostgreSQL with Python
import psycopg2 

### 1. Isolate csv files in cwd<a class="anchor" id="section1"></a>

In [2]:
## Returns contents of CWD
display(os.listdir(os.getcwd()))

['.DS_Store',
 'Customer Contracts$.csv',
 'partTwo.ipynb',
 'customer_contracts.csv',
 'partOne.ipynb',
 '.ipynb_checkpoints',
 'Customer Engagements.csv',
 'Customer Demo.csv']

In [3]:
csv_files = []

for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)
        
print(csv_files)

['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv']


### 2. Create a new directory in cmd<a class="anchor" id="section2"></a>

In [4]:
## Bash command - creates a new folder in the cwd
## -- Example
!mkdir directoryExample

In [5]:
## New directory name
dataset_dir = 'datasets'

## Error handling
try:
    ## Saves the bash command in string format
    mkdir = 'mkdir {0}'.format(dataset_dir)
    print(mkdir)
    ## Runs the bash command string
    os.system(mkdir)
except:
    pass

mkdir datasets


### 3. Move csv files into new directory<a class="anchor" id="section3"></a>

In [6]:
## Bash command - moves a specific file into desired folder
## -- Example
!mv fileName directoryName

mv: rename fileName to directoryName: No such file or directory


In [7]:
print(csv_files)

['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv']


In [8]:
## For loop to run bash command
for file in csv_files:
    ## Saves the bash command in string format
    ## -- NOTE: seperate the file name with single quotes to run command
    mvfile = "mv '{0}' {1}".format(file, dataset_dir)
    print(mvfile)
    ## Runs the bash command string
    os.system(mvfile)

mv 'Customer Contracts$.csv' datasets
mv 'customer_contracts.csv' datasets
mv 'Customer Engagements.csv' datasets
mv 'Customer Demo.csv' datasets


### 4. Create pandas DataFrame from csv file(s)<a class="anchor" id="section4"></a>

In [9]:
## Step 1: Define a pathway to the csv file(s)
data_path = os.getcwd() + '/' + dataset_dir +'/'
print(data_path)

/Users/sam/Desktop/automateDataUpload/datasets/


In [10]:
## Step 2: Read csv files into pandas DataFrames

## Dictionary object to store all DataFrames
df_dict = {}
for file in csv_files:
    ## -- Error handling
    try:
        ## KEY : VALUE
        df_dict[file] = pd.read_csv(data_path + file)
    except UnicodeDecodeError:
        df_dict[file] = pd.read_csv(data_path + file, encoding="ISO-8859-1")
    print(file)

Customer Contracts$.csv
customer_contracts.csv
Customer Engagements.csv
Customer Demo.csv


In [11]:
print(df_dict.keys())

## View a DataFrame 
display(df_dict['Customer Contracts$.csv'].head())

dict_keys(['Customer Contracts$.csv', 'customer_contracts.csv', 'Customer Engagements.csv', 'Customer Demo.csv'])


,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


### 5. Import CSV files to a postgreSQL database - automated script<a class="anchor" id="section5"></a>

In [12]:
for file in csv_files:
    
    DataFrame = df_dict[file]

    ## -- 1. CLEAN TABLE & COLUMN HEADER NAMES
    
    ## Step 1: Cleans file names
    clean_file_name = file.lower().replace('%', '').replace('$', '')\
                          .replace('£', '').replace('€', '').replace(')', '')\
                          .replace(r'(', '').replace('?', '').replace('!', '')\
                          .replace(' ', '_').replace('-', '_').replace(r'/', '_')\
                          .replace('\\', '_')
    
    # print(clean_file_name)
    
    ## Step 2: Removes the .csv extension from the filename
    final_table_name = '{0}'.format(clean_file_name.split('.')[0])
    
    # print(final_table_name)
    
    ## Step 3: Cleans header names
    DataFrame.columns = [x.lower().replace('%', '').replace('$', '')\
                          .replace('£', '').replace('€', '').replace(')', '')\
                          .replace(r'(', '').replace('?', '').replace('!', '')\
                          .replace(' ', '_').replace('-', '_').replace(r'/', '_')\
                          .replace('\\', '_') for x in DataFrame.columns]
    
    # print(DataFrame.columns, '\n')
    
    ## -- 2. REPLACE PANDAS DTYPES WITH SQL DATABASE DTYPES
    
    ## Step 1: Dictionary mapping pandas dtypes to SQL dtypes
    data_type_replacements = {'object': 'varchar', 
                              'float64': 'float',
                              'int64': 'int', 
                              'datetime64': 'timestamp',
                              'timedelta64[ns]': 'varchar'}
    
    ## Step 2: Table schema string
    col_string = ', '.join('{} {}'.format(n, d) for (n, d) in zip(DataFrame.columns,\
                                                                  DataFrame.dtypes.replace(data_type_replacements)))
    # print(col_string)
    
    ## -- 3. OPEN A DATABASE CONNECTION
    
    ## Database login credentials
    host = '####.cfa0pnoy####.eu-west-1.rds.amazonaws.com'
    database_name = 'database_github'
    user = 'postgres'
    password = '####'
    
    ## Step 1: Database details string
    connection_string = "host=%s dbname=%s user=%s password=%s" % (host, database_name, user, password)
    
    # step 2: Establish connection (logs in to database)
    rds_connection = psycopg2.connect(connection_string)
    
    # step 3. Open the connection
    cursor = rds_connection.cursor()
    print('Connected to database')
    
    ## -- 4. WRITE SQL STATEMENTS TO DATABASE
    
    ## Step 1: Drops any tables with the same name
    cursor.execute("DROP TABLE IF EXISTS %s;" % (final_table_name))
    
    ## Step 2: Creates table in the database
    cursor.execute("CREATE TABLE %s (%s)" % (final_table_name, col_string))
    
    ## -- 5. INSERT VALUES INTO SQL TABLE
    
    ## Step 1: Save pandas processed DataFrame to a csv file
    DataFrame.to_csv(file, header=DataFrame.columns, index=False, encoding='utf-8')
    
    ## Step 2: Opens the csv file to open it up in memory (note: file is saved as an object)
    my_file = open(file)
    
    ## Step 3: Upload the csv file to the created table in database
    sql_statement = """
                    COPY %s FROM STDIN WITH
                       CSV
                       HEADER
                       DELIMITER AS ','
                    """
    ## Step 4: Execute this SQL statement with copy_expert method to insert values into SQL table
    cursor.copy_expert(sql=sql_statement %final_table_name, file=my_file)
    
    ## -- TABLE PERMISSIONS --
    
    ## Grant multiple-users access
    cursor.execute('GRANT SELECT ON table %s TO public' % final_table_name)
    
    ## Commit to database
    rds_connection.commit()
    
    ## Close the connection to database
    cursor.close()
    print('Table {0} successfully imported to database'.format(final_table_name))

Connected to database
Table customer_contracts successfully imported to database
Connected to database
Table customer_contracts successfully imported to database
Connected to database
Table customer_engagements successfully imported to database
Connected to database
Table customer_demo successfully imported to database


#### 5-1. Quick reference<a class="anchor" id="section5.1"></a>

In [13]:
## For loop - quick view
for file in csv_files:
    DataFrame = df_dict[file]
    display(DataFrame)

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,8.50,Yes,Yes
7,Airbnb,04-02-2008,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-2005,09-20-2009,11.26,Yes,No


,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes
5,Facebook,04-29-2011,NaN,7.58,Yes,No
6,Apple,10-15-2009,NaN,8.50,Yes,Yes
7,Airbnb,04-02-2008,NaN,9.42,No,No
8,Nest,09-19-2006,NaN,3.00,Yes,Yes
9,Canon,03-07-2005,09-20-2009,11.26,Yes,No


,customer_id,num_of_users,_of_all_employees,sso,launched
0,101,10000,65%,Y,Yes
1,102,23423,80%,N,No
2,103,34556,30%,Y,Yes
3,104,123123,33%,N,No
4,105,19832,36%,Y,Yes
5,106,1243,75%,N,No
6,107,1231,42%,Y,Yes
7,108,12200,90%,N,No
8,109,200,48%,Y,Yes
9,110,1100,51%,N,No


,customer_id,customer_name,employee_count,office_location
0,101,Nike,120000,Oregon
1,102,Reebox,5000,California
2,103,Adidas,8000,CA
3,104,Google,500000,CA
4,105,Amazon,200000,Washington
5,106,Facebook,40002,CA
6,107,Apple,8000,CA
7,108,Airbnb,500000,CA
8,109,Nest,200000,CA
9,110,Canon,40002,NY
